In [3]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import random
from sklearn.metrics import accuracy_score
from multiprocessing import Pool
from functools import reduce
import concurrent.futures as futures
import plotly.graph_objs as go
import plotly.io as pio

In [4]:
df = pd.read_csv('train.csv')
# digits = load_digits()
# x = digits.data
# y = digits.target

# category_mapping = {'A': 1, 'B': 2, 'C': 2,'D':3}

# Map categories to their numerical representation
# df['class'] = df['class'].map(category_mapping)
# category_mapping1 = {'Siit_Pistachio': 0, 'Kirmizi_Pistachio': 1 }

# Map categories to their numerical representation
# df['Class'] = df['Class'].map(category_mapping1)
print(df['price_range'])

0       1
1       2
2       2
3       2
4       1
       ..
1995    0
1996    2
1997    3
1998    0
1999    3
Name: price_range, Length: 2000, dtype: int64


In [5]:
x = df.drop(["price_range"],axis=1)
y = df["price_range"].values.reshape(df.shape[0],1)

scaler = MinMaxScaler()
x = scaler.fit_transform(x)
# x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state=42)


In [6]:
def relu(x):
    return np.maximum(0, x)
def leaky_relu(x, a=0.01):
    return np.maximum(a*x, x)
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()
    
def sigmoid(Z):
    A = 1/(1+np.exp(-Z))
    return A

In [7]:
class Layer:
    def __init__(self, input_units, output_units, activation_function=None):
#         random_array = np.random.uniform(low, high, size=(input_units, output_units))
        self.W = np.random.uniform(-1, 1, size=(input_units, output_units))
        self.b = np.random.uniform(-1, 1, size=(output_units))
        self.activation_function = activation_function

    def forward(self, inputs):
        linear_output = np.dot(inputs, self.W) + self.b
        if self.activation_function is None:
            return linear_output
        else:
            return self.activation_function(linear_output)

class NeuralNetwork:
    def __init__(self):
        self.layers = []
        self.model_weights = []
        self.model_biases = []
        self.accuracy = None

    def add(self, layer):
        self.layers.append(layer)
        self.model_weights.append(layer.W)
        self.model_biases.append(layer.b)
        
#     def predict_prob(self, input_data):
#         results = []
#         for data in input_data:
#             output = data
#             for layer in self.layers:
#                 output = layer.forward(output)
#             results.append(output)
#         return results 
    def predict_prob(self, input_data):
        return [reduce(lambda output, layer: layer.forward(output), self.layers, data) for data in input_data]

    def predict(self, input_data):
        prob_predictions = self.predict_prob(input_data)
        return [np.argmax(prediction) for prediction in prob_predictions]


    def print_architecture(self):
        print("Model Architecture:")
        for i, layer in enumerate(self.layers):
            activation = layer.activation_function.__name__ if layer.activation_function else "None"
            print(f"Layer {i+1}: Units={layer.W.shape[1]}, Activation={activation}")

    def print_weights(self):
        print("Model Weights:")
        for i, w in enumerate(self.model_weights):
            print(f"Layer {i+1} shape : {w.shape} weights:")
            print(w)
            
    def set_accuracy(self, input_data, true_labels):
        predicted_labels = self.predict(input_data)
#         correct_predictions = sum(1 for pred, true in zip(predicted_labels, true_labels) if pred == true)
#         total_predictions = len(true_labels)
#         accuracy_percentage = (correct_predictions / total_predictions) * 100
#         self.accuracy = accuracy_percentage
        self.accuracy= accuracy_score(true_labels,predicted_labels)*100
    def get_accuracy(self):
        return self.accuracy

In [8]:
def gen_individual():
    neural_network = NeuralNetwork()
    first_layer = Layer(20, 32, leaky_relu)
    second_layer = Layer(32, 32, leaky_relu)
    # third_layer = Layer(32, 32, relu)
#     fourth_layer = Layer(16, 16, np.tanh)
    last_layer = Layer(32, 4, softmax)
    neural_network.add(first_layer)
    neural_network.add(second_layer)
#     neural_network.add(third_layer)
#     neural_network.add(fourth_layer)
    neural_network.add(last_layer)
    return neural_network

In [9]:
def euclidean_distance(ind1, ind2):
    total_distance = 0
    for w1, w2, b1, b2 in zip(ind1.model_weights, ind2.model_weights, ind1.model_biases, ind2.model_biases):
        total_distance += np.sum((w1 - w2) ** 2)
        total_distance += np.sum((b1 - b2) ** 2)
    return np.sqrt(total_distance)

In [10]:
def fitness(individual):
    individual.set_accuracy(x, y)
    accuracy = individual.get_accuracy()
    return accuracy

In [11]:
def shared_fitness(individual, population, sigma_share=20.5, alpha=1.0):
    distances = np.array([euclidean_distance(individual, other) for other in population if individual != other])
    sharing_sum = np.sum(1 - (distances[distances < sigma_share] / sigma_share) ** alpha)
#     print(distances)
    raw_fitness = fitness(individual)
    adjusted_fitness = raw_fitness / (1 + sharing_sum)
    return adjusted_fitness


In [12]:
def generate_population(population_size):
    population = []
    fitnesses = []
    for i in range(population_size):
        individual = gen_individual()
        population.append(individual)
        fitnesses.append(fitness(individual))
    return population, fitnesses

In [13]:
# population, fitnesses = generate_population(100)
# shared_fitness(population[0],population)

In [14]:
# shared_fitness(population[0],population)

In [15]:
def new_child(p1, p2, p3, target,cr, f):
    mutant=gen_individual()
    trial=gen_individual()
    for i in range(len(mutant.model_weights)):
      differencew = f * (p1.model_weights[i] - p2.model_weights[i])
      differenceb = f * (p1.model_biases[i] - p2.model_biases[i])
      mutant.model_weights[i]= p3.model_weights[i] + differencew
      mutant.model_biases[i]= p3.model_biases[i]  + differenceb

      for j in range(len(trial.model_weights[i])):
        trial.model_weights[i][j]= mutant.model_weights[i][j] if random.random() <= cr else target.model_weights[i][j]
      for j in range(len(trial.model_biases[i])):
        trial.model_biases[i][j]= mutant.model_biases[i][j] if random.random() <= cr else target.model_biases[i][j]
    return trial
    

In [16]:
def evaluate_target(args):
    target, population, f, cr, sigma_share, alpha = args
    a, b, c = random.sample(list(population), 3)
    trial = new_child(a, b, c, target, cr, f)
#     trial_acc = fitness(trial)
#     target_acc = fitness(target)
    trial_acc = shared_fitness(trial, population, sigma_share, alpha)
    target_acc = shared_fitness(target, population, sigma_share, alpha)
    if trial_acc > target_acc:
        result = [trial, trial_acc]
    else:
        result = [target, target_acc]
    return result

In [18]:
def differential_evolution(population_size, f=1.5, cr=0.9, max_iters=10, sigma_share=130.0, alpha=1.0, max_stall_iters=15, percentage_to_keep=0.25):
    # Initialize history dictionary to store accuracies
    history = {
        "best": [],
        "average": [],
        "worst": []
    }
    population, fitnesses = generate_population(population_size)
    best = population[np.argmax(fitnesses)]
    best_acc = max(fitnesses)
    print(best_acc)
    num_iters = 0
    stall_iters = 0
    while num_iters < max_iters:
        with futures.ThreadPoolExecutor(max_workers=12) as executor:
            args_list = [(target, population, f, cr, sigma_share, alpha) for target in population]
            try:
                results = list(executor.map(evaluate_target, args_list))
            except Exception as e:
                print(f"Error: {e}")
                break
  
        results = np.array(results)
        
        new_population = results[:,0]
        new_fitnesses = results[:,1]



        best_index = np.argmax(new_fitnesses)
        new_best_acc = new_fitnesses[best_index]

        if new_best_acc > best_acc:
            best_acc = new_best_acc
            best = new_population[best_index]
            stall_iters = 0
        else:
            stall_iters += 1

        population = new_population
        num_iters += 1
        avr=np.mean(new_fitnesses)

        # Store accuracies in history dictionary
        history["best"].append(best_acc)
        history["average"].append(avr)
        history["worst"].append(np.min(new_fitnesses))

        print(f"Iteration: {num_iters} -> Best Accuracy: {best_acc:.2f} % -> Average:{avr:.2f} ")

        # Restart if best accuracy has not improved for max_stall_iters iterations
        if stall_iters == max_stall_iters:
            print(f"Restarting population due to stalling for {max_stall_iters} iterations...")
            sorted_indices = np.argsort(-new_fitnesses)
            keep = int(percentage_to_keep*population_size)
            new_indx = sorted_indices[:keep]
            keep_pop = new_population[new_indx]
            population, fitnesses = generate_population(population_size-keep)
            # population.append(best)
            # fitnesses.append(best_acc)
            population = np.concatenate((keep_pop, population))
            best = population[np.argmax(fitnesses)]
            best_acc = max(fitnesses)
            stall_iters = 0

    return population, best, best_acc, history

In [19]:
def plot_history(history):
    fig = go.Figure()

    for key in history.keys():
        fig.add_trace(go.Scatter(x=list(range(1, len(history[key]) + 1)), y=history[key], mode='lines', name=key))

    fig.update_layout(title='Evolution of Accuracy',
                      xaxis_title='Iteration',
                      yaxis_title='Accuracy',
                      legend_title='Accuracy Type')

    pio.show(fig)

In [20]:
import time
start_time = time.time()
population,best_solution,best_acc, history = differential_evolution(30)
end_time = time.time()
print((end_time - start_time)/60)

31.05
Iteration: 1 -> Best Accuracy: 31.05 % -> Average:1.46 
Iteration: 2 -> Best Accuracy: 31.05 % -> Average:13.58 
Iteration: 3 -> Best Accuracy: 32.85 % -> Average:26.08 
Iteration: 4 -> Best Accuracy: 34.40 % -> Average:27.24 
Iteration: 5 -> Best Accuracy: 34.40 % -> Average:27.57 
Iteration: 6 -> Best Accuracy: 34.40 % -> Average:28.33 
Iteration: 7 -> Best Accuracy: 34.40 % -> Average:28.63 
Iteration: 8 -> Best Accuracy: 34.40 % -> Average:28.75 
Iteration: 9 -> Best Accuracy: 34.40 % -> Average:29.07 
Iteration: 10 -> Best Accuracy: 35.80 % -> Average:29.76 
1.3400258421897888


In [21]:
plot_history(history)